<a href="https://colab.research.google.com/github/bedoo96/Pre-training-LLM-HuggingFace-Pytorch-/blob/main/Pre-training_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Objectives</h1>

<ol>
  <li>Load pretrained LLMs from HuggingFace and make inferences using pipeline module</li>
  <li>Train pretrained on my own data</li>
  <li>Stores LLMs for fine-tune on specific use case</li>
</ol>

In [1]:
# # installing librairies

# ! pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch=2.1.0+cu118
# # ! pip install pmdarima -U
# ! pip install --upgrade pmdarima==2.0.2

In [2]:
# !pip install transformers==4.40.0
# ! pip install -U git+https://github.com/huggingface/transformers
# ! pip install --user datasets # 2.15.0
# ! pip install --user portalocker>=2/0.0
# ! pip install -q -U git+https://github.com/huggingface/accelerate.git
# ! pip install --user torch==2.3.0
# ! pip install -U torchvision
# ! pip install --user protobuf==3.20.*


In [3]:
! pip install  datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00


In [4]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [5]:
# set the environment variable tokenizer_parallelism to false
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

<h1>Pretraining and self-supervised fine-tuning</h1>

In [6]:
# Let's load pretraining model from hugging face
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

pipe = pipeline("text-generation", model=model,tokenizer=tokenizer)
print(pipe("This movie was really")[0]["generated_text"])

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Device set to use cuda:0


This movie was really good. I was really surprised by how good it was.
I was


<h2>Self-supervised training of BERT model</h2>

<h4>Objectives</h4>

<ol>
  <li>Prepare the train dataset</li>
  <li>Train a Tokenizer</li>
  <li>Preprocess the dataset</li>
  <li>Pre-train BERT using an MLM task</li>
  <li>Evaluate the trained model</li>

</ol>

In [7]:
# Load the datasets
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [9]:
dataset["train"] = dataset["train"].select([i for i in range(1000)])
dataset["test"] = dataset["test"].select([i for i in range(200)])

print(dataset["train"])
print(dataset["test"])

Dataset({
    features: ['text'],
    num_rows: 1000
})
Dataset({
    features: ['text'],
    num_rows: 200
})


In [10]:
# Path to save the datasets to text files
output_file_train = "wikitext_dataset_train.txt"
output_file_test = "wikitext_dataset_test.txt"

# Open the output file in write mode
with open(output_file_train, "w", encoding="utf-8") as f:
    # Iterate over each example in the dataset
    for example in dataset["train"]:
        # Write the example text to the file
        f.write(example["text"] + "\n")

# Open the output file in write mode
with open(output_file_test, "w", encoding="utf-8") as f:
    # Iterate over each example in the dataset
    for example in dataset["test"]:
        # Write the example text to the file
        f.write(example["text"] + "\n")


In [11]:
# create a tokenizer from existing one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
model_name = 'bert-base-uncased'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, is_decoder=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [13]:
## create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    for i in tqdm(range(0, len(dataset), batch_size)):
        yield dataset['train'][i : i + batch_size]["text"]

## create a tokenizer from existing one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## train the tokenizer using our own dataset
bert_tokenizer = bert_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=30522)

  0%|          | 0/1 [00:00<?, ?it/s]

<h4>Pretraining</h4>

<h5>BERT configuration</h5>
- <b>vocab_size=30522:</b> Specifies the size of the vocabulary. This number should match the vocabulary size used by the tokenizer.

<b>hidden_size=768:</b> Sets the size of the hidden layers.

<b>num_hidden_layers=12:</b> Determines the number of hidden layers in the transformer model.

<b>num_attention_heads=12:</b> Sets the number of attention heads in each attention layer.

<b>intermediate_size=3072:</b> Specifies the size of the "intermediate" (i.e., feed-forward) layer within the transformer.

In [14]:
# Define the BERT configuration
config = BertConfig(
    vocab_size=len(bert_tokenizer.get_vocab()),  # Specify the vocabulary size(Make sure this number equals the vocab_size of the tokenizer)
    hidden_size=768,  # Set the hidden size
    num_hidden_layers=12,  # Set the number of layers
    num_attention_heads=12,  # Set the number of attention heads
    intermediate_size=3072,  # Set the intermediate size
)

In [15]:
# Create the BERT model for pre-training
model = BertForMaskedLM(config)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [16]:
# check model configuration
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(12576, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

<h4>Define the training Dataset</h4>

In [17]:
# Prepare the pre-training data as a TextDataset
train_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_train.txt",  # Path to your pre-training data file
    block_size=128  # Set the desired block size for training
)
test_dataset = TextDataset(
    tokenizer=bert_tokenizer,
    file_path="wikitext_dataset_test.txt",  # Path to your pre-training data file
    block_size=128  # Set the desired block size for training
)

Token indices sequence length is longer than the specified maximum sequence length for this model (55468 > 512). Running this sequence through the model will result in indexing errors


In [18]:
train_dataset[0]

tensor([    2,    31,   619,   799,  1414,    31,  3542,   593,   619,    22,
           29, 12314,   799,    12,  3189,    29,   105,   103,  5411,    15,
          490,    17,   619,   184,   171,  2553,    22,    13,    15,  3534,
         3584,   191,   216,   619,   799,  1414,  1540,  2033,    15,   256,
           35,  5440,  1124,    32,    16,    32,  1843,  2188,   398,  1762,
          251,  3403,   188,  3160,    17,  2434,   209,   171,  2290,  4153,
           17,  1349,   180,  1685,  1213,   180,  2033,    15,   288,   256,
          171,  1310,   398,   180,   171,   619,  1220,    17,  6799,   171,
          859,  7535,   184,  5440,   188,  1151,    32,    16,    32,   533,
         1972,   216,   408,  4372,    15,   171,  1504, 10326,  9717,   191,
          171,   447,   398,   188,  6351,   171,     6,  1864,     6,    15,
           35,  3528,  1062,  2489,  4113,   171,  1807,   184,  3575,   514,
          171,   753,  7205,   527,   407,   852,  2471,     3])

In [19]:
# Prepare the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)

In [20]:
# check how collator transforms a sample input data record
data_collator([train_dataset[0]])

{'input_ids': tensor([[    2,    31,     4,   799,  1414,    31,  3542,   593,     4,    22,
             29, 12314,   799,    12,  3189,    29,   105,   103,  5411,    15,
            490,    17,   619,   184,   171,  2553, 12474,    13,    15,  3534,
           3584,   191,   216,   619,   799,  1414,  1540,     4,    15,   256,
             35,  5440,  1124,    32,    16,     4,  1843,  2188,   398,  1762,
            251,  3403,     4,  3160,    17,  2434,   209,   171,  2290,     4,
             17,  1349,   180,  1685,  1213,   180,  2033,    15,   288,     4,
            171,  1310,   398,   180,   171,   619,     4,    17,  6799,   171,
            859,     4,   184,  5440,   188,  1151,     4,    16,    32,   533,
           1972,   216,   408,  4372,    15,   171,  1504, 10326,  9717,   191,
            171,   447,   398,     4,  6351,   171,     6,  6372,     6,    15,
             35,  3528,  1062,  2489,  4113,     4,  1807,   184,     4,   514,
            171,   753,    

<h3> Training</h3>

In [31]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./trained_model",  # Specify the output directory for the trained model
    overwrite_output_dir=True,
    do_eval=True,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    num_train_epochs=20,  # Specify the number of training epochs
    per_device_train_batch_size=4,  # Set the batch size for training
    save_total_limit=2,  # Limit the total number of saved checkpoints
    logging_steps = 20

)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start the pre-training
trainer.train()
# Save the tokenizer explicitly
tokenizer.save_pretrained("./trained_model")

Epoch,Training Loss,Validation Loss
1,6.634700,8.879608
2,6.838700,8.793407
3,6.668100,8.742157
4,6.682100,8.851356
5,6.602300,8.901476
6,6.657200,8.602174
7,6.594500,8.749797
8,6.552900,8.749346
9,6.678700,8.699375
10,6.748600,8.898298


('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [32]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6227.60


In [33]:
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [46]:
import json

with open("./trained_model/checkpoint-2200/config.json", "r") as f:
    config = json.load(f)

print(config)

{'architectures': ['BertForMaskedLM'], 'attention_probs_dropout_prob': 0.1, 'classifier_dropout': None, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'layer_norm_eps': 1e-12, 'max_position_embeddings': 512, 'model_type': 'bert', 'num_attention_heads': 12, 'num_hidden_layers': 12, 'pad_token_id': 0, 'position_embedding_type': 'absolute', 'torch_dtype': 'float32', 'transformers_version': '4.48.3', 'type_vocab_size': 2, 'use_cache': True, 'vocab_size': 12576}


In [43]:
# loading the saved model
from transformers import AutoModel, AutoTokenizer

# Load the model
model = AutoModel.from_pretrained("./trained_model/checkpoint-2200")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("./trained_model")

Some weights of BertModel were not initialized from the model checkpoint at ./trained_model/checkpoint-2200 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
from transformers import BertForMaskedLM, AutoTokenizer, pipeline

# Create the fill-mask pipeline
mask_filler = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Generate predictions
results = mask_filler(text)

# Print the predicted sequences
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

Device set to use cuda:0


Predicted token: [unused166], Confidence: 0.05
Predicted token: [unused16], Confidence: 0.04
Predicted token: [unused14], Confidence: 0.04
Predicted token: [unused179], Confidence: 0.02
Predicted token: [unused183], Confidence: 0.02


<b> that [MASK] is replaced by the most frequent token. This weak performance can be due to insufficient training, lack of training data, model architecture, or not tuning hyperparameters</b>

In [36]:
# Load the pretrained BERT model and tokenizer
pretrained_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
pretrained_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Create the pipeline
mask_filler = pipeline(task='fill-mask', model=pretrained_model,tokenizer=pretrained_tokenizer)

# Perform inference using the pipeline
results = mask_filler(text)
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Predicted token: great, Confidence: 0.16
Predicted token: horror, Confidence: 0.08
Predicted token: good, Confidence: 0.08
Predicted token: bad, Confidence: 0.05
Predicted token: fantastic, Confidence: 0.04
